In [1]:
#!pip install contextily
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import folium
import pandas as pd
import urllib3
from zipfile import ZipFile
import requests

In [8]:
# Téléchargement :
# https://download.geofabrik.de/europe/france/ile-de-france.html
data_folder = "./data/"
zipname = "Ile-de-France-Geofabrik-free"
geofabrik_idf_shp_link = "https://download.geofabrik.de/europe/france/ile-de-france-latest-free.shp.zip"
r = requests.get(geofabrik_idf_shp_link)

open(data_folder+zipname+".zip",'wb').write(r.content)


484405588

In [20]:
# unzip des données en provenance de :
geofabrik_folder = data_folder+zipname+"/"
datafile_name = 'gis_osm_pois_a_free_1.shp'
dataextensions_unzip = ['.shp','.shx','.cpg','.dbf','.prj']
with ZipFile(data_folder+zipname+".zip") as myzip:
    for ext in dataextensions_unzip :
        myzip.extract(datafile_name[:-4]+ext, path = geofabrik_folder)

In [21]:
# ouverture des données
df_pois = gpd.read_file(geofabrik_folder+datafile_name)

# note : pois stands for points of interests

In [22]:
"""
I Only keep leisure and some shops and markets (see documentation : https://download.geofabrik.de/osm-data-in-gis-formats-free.pdf) 
"""

df_pois  = df_pois[(df_pois["code"] >= 2201) 
                   & (df_pois["code"] <= 2706)]

In [23]:

df_pois = df_pois.reset_index(drop = True) #I restart index  

In [24]:
df_pois 

,osm_id,code,fclass,name,geometry
0,4083189,2204,park,Parc de Bercy,"POLYGON ((2.37807 48.83739, 2.37812 48.83741, ..."
1,4208595,2204,park,Champ de Mars,"POLYGON ((2.29187 48.85771, 2.29230 48.85801, ..."
2,4221369,2204,park,Jardin des Plantes,"POLYGON ((2.35510 48.84374, 2.35511 48.84375, ..."
3,4263203,2204,park,Jardin du Palais Royal,"POLYGON ((2.33672 48.86417, 2.33674 48.86422, ..."
4,4410586,2204,park,None,"POLYGON ((2.20680 48.87370, 2.20770 48.87409, ..."
...,...,...,...,...,...
46023,1112119036,2421,shelter,None,"POLYGON ((2.46203 48.81455, 2.46204 48.81456, ..."
46024,1112121750,2421,shelter,None,"POLYGON ((2.41404 48.82155, 2.41410 48.82157, ..."
46025,1112127658,2421,shelter,None,"POLYGON ((2.41030 48.82361, 2.41032 48.82362, ..."
46026,1112127661,2421,shelter,None,"POLYGON ((2.40987 48.82409, 2.40989 48.82410, ..."


In [25]:
"""
I use folium to create an interative map centered in Eiffel Tower
"""


map = folium.Map(location=[48.857551, 2.29394], tiles="OpenStreetMap", zoom_start=14)
map

In [26]:
"""
I use gpd.centroid to have polygon centers (more easy to manipulate) 
"""
# geometry (active) column
df_pois = df_pois.to_crs(epsg=4326)
df_pois['center'] = df_pois.centroid
# Centroid column
df_pois["center"] = df_pois['center'].to_crs(epsg=4326)

df_pois.head()

C:\Users\leopo\AppData\Local\Temp\ipykernel_19944\2415629637.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_pois['center'] = df_pois.centroid


,osm_id,code,fclass,name,geometry,center
0,4083189,2204,park,Parc de Bercy,"POLYGON ((2.37807 48.83739, 2.37812 48.83741, ...",POINT (2.38212 48.83549)
1,4208595,2204,park,Champ de Mars,"POLYGON ((2.29187 48.85771, 2.29230 48.85801, ...",POINT (2.29792 48.85605)
2,4221369,2204,park,Jardin des Plantes,"POLYGON ((2.35510 48.84374, 2.35511 48.84375, ...",POINT (2.35926 48.84334)
3,4263203,2204,park,Jardin du Palais Royal,"POLYGON ((2.33672 48.86417, 2.33674 48.86422, ...",POINT (2.33781 48.86501)
4,4410586,2204,park,None,"POLYGON ((2.20680 48.87370, 2.20770 48.87409, ...",POINT (2.20915 48.87240)


In [27]:
"""
I filter a zone near Eiffel Tower. 
"""

gdf_pois = pd.concat([df_pois, df_pois.bounds], axis=1) #I use gpd bounds to have coordonnees in two columns and not only in Point(x,y) 

gdf_pois_eiffel = gdf_pois[(gdf_pois["minx"] >= 2.287592 ) &
        (gdf_pois["minx"] <= 2.314801) &
         (gdf_pois["miny"] >= 48.847194) &
        (gdf_pois["miny"] <= 48.866217)]

gdf_pois_eiffel = gdf_pois_eiffel.reset_index(drop = True) #reset index. 

In [28]:
"""
It is the classical folium method : you create a list with geometry point values (here gdf_pois_eiffel.center) and fir each 
coordinates, you add a marker. 
"""

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf_pois_eiffel.center] 


i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup=
                 "Name: " + str(gdf_pois_eiffel.name[i]) + "<br>" #here to add name 
                + "type: " + str(gdf_pois_eiffel.fclass[i]) + "<br>" #type 
                + "Coordinates: " + str(geo_df_list[i])
            ,
        
            icon=folium.Icon(color="blue"),
        )
    )
    i = i + 1

In [29]:
map